In [57]:
import argparse
import logging
import os
from pathlib import Path

import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

import matplotlib.pyplot as plt
from SeaIce.unet.network_structure import UNet
from SeaIce.unet.dataset_preparation import create_npy_list
from SeaIce.unet.shared import *
import segmentation_models_pytorch as smp
#import SeaIce.unet.model_pretrained
from sklearn.metrics import precision_score, accuracy_score


def load_model(model_path, unet_type, image_type):
    if image_type == "modis":
        channels = 3
    elif image_type == "sar":
        channels = 1
    if unet_type == "raw":
        model = UNet(n_channels=channels, n_classes=2, bilinear=True)
    elif unet_type == "pretrained":
        model = 0
        model = smp.Unet(encoder_name='resnet34', encoder_weights='imagenet', decoder_use_batchnorm=True,
                 decoder_attention_type=None, in_channels=channels, classes=1, encoder_depth=5)
        model = model.double()
    state = torch.load(model_path)
    model.load_state_dict(state)
    return(model)


def plot_img_and_mask(img, mask, image_type):
    classes = mask.shape[0] if len(mask.shape) > 2 else 1
    fig, ax = plt.subplots(1, classes + 1)
    ax[0].set_title('Input image')
    if image_type == "sar":
            ax[0].imshow(img, cmap='gray')
    elif image_type == "modis":
            ax[0].imshow(img)
    if classes > 1:
        for i in range(classes):
            ax[i + 1].set_title(f'Output mask (class {i + 1})')
            ax[i + 1].imshow(mask[i, :, :])
    else:
        ax[1].set_title(f'Output mask')
        ax[1].imshow(mask)
    plt.xticks([]), plt.yticks([])
    plt.show()
    

def unlog_img(img):
    img_e = np.exp(np.array(img))
    reutrn(img_e)
    

def predict_img(net,
                full_img,
                device,
                scale_factor=1,
                out_threshold=0.5):
    net.eval()
    img = full_img 
    img = img.unsqueeze(0)
    img = img.to(device=device, dtype=torch.float32)

    with torch.no_grad():
        output = net(img)

        if net.n_classes > 1:
            probs = F.softmax(output, dim=1)[0]
        else:
            probs = torch.sigmoid(output)[0]

        tf = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor()
        ])

        full_mask = tf(probs.cpu()).squeeze()

    if net.n_classes == 1:
        return (full_mask > out_threshold).numpy()
    else:
        return F.one_hot(full_mask.argmax(dim=0), net.n_classes).permute(2, 0, 1).numpy()


def mask_to_image(mask: np.ndarray):
    if mask.ndim == 2:
        return Image.fromarray((mask * 255).astype(np.uint8))
    elif mask.ndim == 3:
        return Image.fromarray((np.argmax(mask, axis=0) * 255 / mask.shape[0]).astype(np.uint8))
    

def make_predictions(model_path, unet_type, image_type, dir_test, dir_out, log = False, metrics = False, viz = False, save = False):
    img_list = create_npy_list(dir_test, image_type)
    
    net = load_model(model_path, unet_type, image_type)

    for i in enumerate(img_list):
        filename = i[1][0]
        if metrics == True:
            groundtruth_filename = filename.replace(image_type,'labels')
            gt_npy = np.vstack(np.load(groundtruth_filename))
            gt100 = np.where(gt_npy == 100, 0, gt_npy)
            gt200 = np.where(gt100 == 200, 1, gt100)
            gt_remap = np.vstack(gt200).astype(int)
            #print(gt_remap)
        logging.info(f'\nPredicting image {filename} ...')
        if log: npimg = unlog_img(np.load(filename))
        else: npimg = np.load(filename)
        img = torch.from_numpy(np.vstack(npimg).astype(int))
        if image_type == "sar":
            img = img[None,:]
        elif image_type == "modis":
            img = torch.permute(img, (2, 0, 1))
        
        mask = predict_img(net=net,
                            full_img=img,
                            scale_factor=0.5,
                            out_threshold=0.5,
                            device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        
        if metrics:
            #print(gt_remap.shape, mask[0].shape)
            #print(np.unique(gt_remap), np.unique(mask[0]))
            #print(accuracy_score(gt_remap, mask[0], average = 'micro'), precision_score(gt_remap, mask[0]))
            N = gt_remap.shape[0] * gt_remap.shape[1]
            #accuracy = (gt_remap == mask).sum() / N
            TP = ((mask[0] == 1) & (gt_remap == 1)).sum()
            TN = ((mask[0] == 0) & (gt_remap == 0)).sum()
            FP = ((mask[0] == 1) & (gt_remap == 0)).sum()
            FN = ((mask[0] == 0) & (gt_remap == 1)).sum()
            precision = TP / (TP+FP)
            recall = TP / (TP+FN)
            accuracy = (TP+TN) / N
            print("Precision: {}, Recall: {}, Accuracy: {}".format(precision, recall, accuracy))
        
        if viz:
             logging.info(f'Visualizing results for image {filename}, close to continue...')
             plot_img_and_mask(img.squeeze(), mask, image_type)

        if save:
             out_filename = str(dir_out) + "/" + str(filename[-33:-4]) + "_prediction.png"
             print(out_filename)
             result = mask_to_image(mask)
             result.save(out_filename)
             logging.info(f'Mask saved to {out_filename}')

                
#model_path = Path('/mnt/g/Shared drives/2021-gtc-sea-ice/model/checkpoints/unet_orig/checkpoint_epoch1.pth')
#unet_type = 'raw'
#image_type = 'sar'
#dir_test = Path('/mnt/g/Shared drives/2021-gtc-sea-ice/trainingdata/testing/')
#dir_out = Path('/mnt/g/Shared drives/2021-gtc-sea-ice/model/outtiles/')

#test = make_predictions(model_path, unet_type, image_type, dir_test, dir_out, viz = True, save = False)

In [58]:
model_path = Path('/mnt/g/Shared drives/2021-gtc-sea-ice/model/checkpoints/unet_orig_batch10/checkpoint_epoch1.pth')
unet_type = 'raw'
image_type = 'sar'
dir_test = Path('/mnt/g/Shared drives/2021-gtc-sea-ice/trainingdata/testing/')
dir_out = Path('/mnt/g/Shared drives/2021-gtc-sea-ice/model/outtiles/')

make_predictions(model_path, unet_type, image_type, dir_test, dir_out, log = False, metrics=True, viz = False, save = False)

/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/site-packages/ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in long_scalars


Precision: nan, Recall: 0.0, Accuracy: 0.7608070373535156
Precision: nan, Recall: 0.0, Accuracy: 0.41857147216796875
Precision: nan, Recall: 0.0, Accuracy: 0.37485504150390625
Precision: nan, Recall: 0.0, Accuracy: 0.4349861145019531
Precision: 1.0, Recall: 0.026686217008797655, Accuracy: 0.9873390197753906
Precision: 0.9460070611073779, Recall: 0.45104839981081507, Accuracy: 0.5689239501953125
